In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/"
!git clone https://github.com/ultralytics/yolov5.git
%cd "/content/drive/MyDrive/yolov5/"
!pip install -r requirements.txt

In [ ]:
import os
import natsort
from glob import glob
from sklearn.model_selection import train_test_split

img_list = glob('/content/drive/MyDrive/yolov5/data/images/*.jpg')
print(len(img_list))

img_list = natsort.natsorted(img_list)
train_img_list, val_img_list = train_test_split(img_list, test_size=0.3, random_state=0)
print(len(train_img_list), len(val_img_list))

1347
942 405


In [ ]:
with open('/content/drive/MyDrive/yolov5/data/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/yolov5/data/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml
with open('/content/drive/MyDrive/yolov5/pinkladydata.yaml', 'w') as f:
  f.write("nc: \n")
  f.write("names: \n")
  f.write("train: \n")
  f.write("val: \n")

with open('/content/drive/MyDrive/yolov5/pinkladydata.yaml', 'r') as f:
  try:
    data = yaml.load(f)
  except:
    data = yaml.full_load(f)

print(data)

data["nc"] = 1
data["names"] = ["pinklady"]
data['train'] = "/content/drive/MyDrive/yolov5/data/train.txt"
data['val'] = "/content/drive/MyDrive/yolov5/data/val.txt"

with open('/content/drive/MyDrive/yolov5/pinkladydata.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'nc': None, 'names': None, 'train': None, 'val': None}
{'nc': 1, 'names': ['pinklady'], 'train': '/content/drive/MyDrive/yolov5/data/train.txt', 'val': '/content/drive/MyDrive/yolov5/data/val.txt'}


In [ ]:
# use before using pre-trained weights
!python -c "from utils.general import *; strip_optimizer('/content/drive/MyDrive/yolov5/crowdhuman_yolov5m.pt')"

Optimizer stripped from /content/drive/MyDrive/yolov5/crowdhuman_yolov5m.pt, 42.4MB


In [ ]:
%cd "/content/drive/MyDrive/yolov5/"
!python train.py \
--img 640 \
--batch 40 \
--epochs 77 \
--data "./pinkladydata.yaml" \
--optimizer "AdamW" \
--weights crowdhuman_yolov5m.pt \
--name crowdhuman_yolov5m

/content/drive/MyDrive/yolov5
train: weights=crowdhuman_yolov5m.pt, cfg=, data=./pinkladydata.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=77, batch_size=40, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=runs/train, name=crowdhuman_yolov5m, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 5 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr

In [ ]:
# 실시간으로 안 보고 바로 파일로 저장
%cd "/Users/hch/dopcv/yolov5/"
!python detect.py \
--weights './weights/ft_d_b-40_e-77/weights/best.pt' \
--img 640 \
--conf 0.5 \
--name 'result_ft_d_b-40_e-77_please' \
--device "gpu" \
--source '/project/dl_pro1/original_data/Multi-view_tracking_indoor/00001_market/videos/04.mp4' \
--save-crop

In [ ]:
# 재밌는거

from pathlib import Path

# prefix components:
space =  '    '
branch = '│   '
# pointers:
tee =    '├── '
last =   '└── '


def tree(dir_path: Path, prefix: str=''):
    """A recursive generator, given a directory Path object
    will yield a visual tree structure line by line
    with each line prefixed by the same characters
    """    
    contents = list(dir_path.iterdir())
    # contents each get pointers that are ├── with a final └── :
    pointers = [tee] * (len(contents) - 1) + [last]
    for pointer, path in zip(pointers, contents):
        yield prefix + pointer + path.name
        if path.is_dir(): # extend the prefix and recurse:
            extension = branch if pointer == tee else space 
            # i.e. space because last, └── , above so no more |
            yield from tree(path, prefix=prefix+extension)

# for line in tree(Path.home() / 'pyscratch'):
#    print(line)

target_folder_dir = "/content/drive/MyDrive"
for line in tree(Path(target_folder_dir)):
    print(line)